# 14.8 来自Transformers的双向编码器表示（BERT）
- **目录**
  - 14.8.1 从上下文无关到上下文敏感
  - 14.8.2 从特定任务到不可知任务
  - 14.8.3 BERT：把两个最好的结合起来
  - 14.8.4 输入表示
  - 14.8.5 预训练任务
    - 14.8.5.1 掩蔽语言模型
    - 14.8.5.2 下一句预测

- 前面章节已经介绍了几种用于自然语言理解的词嵌入模型
- 在预训练之后，输出可以被认为是一个矩阵，其中每一行都是一个表示预定义词表中词的向量。
- 事实上，这些词嵌入模型都是与上下文无关的，让我们先来说明这个性质。

----------
- **说明：为何说词嵌入模型都是与上下文无关的？**
  - 该结论表明：不论一个词在不同的句子中是如何使用的，它都将有相同的向量表示。
  - **词嵌入模型的定义**:
    - 词嵌入模型通过学习将词映射到高维空间的向量来为词表中的每个词提供一个向量表示。
    - 预训练完成后，可以从模型中获取任何词的向量表示。
  - . **与上下文无关的特点**:
    - 对于传统的词嵌入模型（如word2vec或GloVe），每个词都有一个固定的向量表示。
    - 这意味着，不管词如何使用，其向量表示都是不变的。
  - . **举例**:
    - 考虑词“bank”。这个词在英语中有多个意思。
        1. “I went to the bank to withdraw some money.” (我去银行取了些钱。)
        2. “He sat by the bank of the river.” (他坐在河岸边。)
    - 在第一个句子中，“bank”指的是金融机构；而在第二个句子中，它指的是河的边缘。
    - 但在传统的与上下文无关的词嵌入模型中，无论“bank”出现在哪个句子中，它的向量表示都是相同的。
  - **局限性**:
    - 由于这种模型不考虑上下文，所以它不能捕获像“bank”这样的多义词的所有含义。
    - 这导致了上下文敏感词嵌入模型（如ELMo、BERT等）的发展，这些模型可以根据上下文为词提供不同的向量表示。

----------

## 14.8.1 从上下文无关到上下文敏感

回想一下 14.4节和 14.7节中的实验。例如，word2vec和GloVe都将相同的预训练向量分配给同一个词，而不考虑词的上下文（如果有的话）。形式上，任何词元$x$的上下文无关表示是函数$f(x)$，其仅将$x$作为其输入。考虑到自然语言中丰富的多义现象和复杂的语义，上下文无关表示具有明显的局限性。例如，在“a crane is flying”（一只鹤在飞）和“a crane driver came”（一名吊车司机来了）的上下文中，“crane”一词有完全不同的含义；因此，**同一个词可以根据上下文被赋予不同的表示**。

这推动了“**上下文敏感**”词表示的发展，其中词的表征取决于它们的上下文。因此，词元$x$的上下文敏感表示是函数$f(x, c(x))$，其取决于$x$及其上下文$c(x)$。流行的上下文敏感表示包括TagLM（language-model-augmented sequence tagger，语言模型增强的序列标记器）、CoVe（Context Vectors，上下文向量）和ELMo（Embeddings from Language Models，来自语言模型的嵌入）。

例如，通过将整个序列作为输入，ELMo是为输入序列中的每个单词分配一个表示的函数。具体来说，**ELMo将来自预训练的双向长短期记忆网络的所有中间层表示组合为输出表示**。然后，ELMo的表示将作为附加特征添加到下游任务的现有监督模型中，例如通过将ELMo的表示和现有模型中词元的原始表示（例如GloVe）连结起来。一方面，在加入ELMo表示后，冻结了预训练的双向LSTM模型中的所有权重。另一方面，现有的监督模型是专门为给定的任务定制的。利用当时不同任务的不同最佳模型，添加ELMo改进了六种自然语言处理任务的技术水平：情感分析、自然语言推断、语义角色标注、共指消解、命名实体识别和问答。
- **要点：**
  - **上下文无关与上下文敏感**:
    - word2vec和GloVe为同一词分配相同的预训练向量，不考虑上下文。这是上下文无关的表示。
    - 由于多义词和复杂的语义，上下文无关表示具有局限性。例如，"crane"在不同的句子中有不同的意义。
    - 这推动了上下文敏感词表示的发展，其中词的表示取决于其上下文。
  - **上下文敏感的表示**:
    - 上下文敏感表示的词元是一个函数，它依赖于词元及其上下文。
    - 流行的上下文敏感表示有TagLM、CoVe和ELMo。
  - **ELMo的特性**:
    - ELMo为输入序列中的每个单词分配表示，将整个序列作为输入。
    - 它从预训练的双向LSTM的所有中间层获得表示，并组合这些表示作为输出。
    - ELMo的表示被加入到下游任务的监督模型中，作为附加特征，与原始表示（如GloVe）连结。
    - 预训练的双向LSTM模型在加入ELMo表示后是冻结的。
    - 现有的监督模型是为特定任务定制的。
  - **ELMo的应用**:
    - 添加ELMo改进了六种自然语言处理任务的技术水平，包括情感分析、自然语言推断、语义角色标注、共指消解、命名实体识别和问答。

## 14.8.2 从特定任务到不可知任务

尽管ELMo显著改进了各种自然语言处理任务的解决方案，但每个解决方案仍然依赖于一个特定于任务的架构。然而，为每一个自然语言处理任务设计一个特定的架构实际上并不是一件容易的事。GPT（Generative Pre Training，生成式预训练）模型为上下文的敏感表示设计了通用的任务无关模型。GPT建立在Transformer解码器的基础上，预训练了一个用于表示文本序列的语言模型。当将GPT应用于下游任务时，语言模型的输出将被送到一个附加的线性输出层，以预测任务的标签。与ELMo冻结预训练模型的参数不同，GPT在下游任务的监督学习过程中对预训练Transformer解码器中的所有参数进行微调。GPT在自然语言推断、问答、句子相似性和分类等12项任务上进行了评估，并在对模型架构进行最小更改的情况下改善了其中9项任务的最新水平。

然而，由于语言模型的自回归特性，GPT只能向前看（从左到右）。在“i went to the bank to deposit cash”（我去银行存现金）和“i went to the bank to sit down”（我去河岸边坐下）的上下文中，由于“bank”对其左边的上下文敏感，GPT将返回“bank”的相同表示，尽管它有不同的含义。

- **要点：**
  - **ELMo的局限性**：
    - ELMo在多种自然语言处理任务上都取得了显著的进展。
    - 但每个解决方案都依赖于特定于任务的架构。
    - 为每个自然语言处理任务设计一个特定的架构是困难的。
  - **GPT模型的特点**：
    - GPT提供了一个通用的、与任务无关的模型来为上下文敏感表示设计。
    - 基于Transformer解码器，预训练了一个表示文本序列的语言模型。
    - 在下游任务中，GPT的输出被送到一个额外的线性输出层来预测任务的标签。
    - 与ELMo不同，GPT在下游任务的监督学习过程中对所有预训练的Transformer解码器的参数进行了微调。
    - GPT在12项任务上进行了评估，对模型架构进行最小的更改并在其中9项任务上提高了最新的水平。
  - **GPT的局限性**：
    - 由于语言模型的自回归特性，GPT只能向前看（从左到右）。
    - 在某些情况下，GPT可能会为有不同含义的词返回相同的表示，例如“bank”的例子中所示。

---------
- **说明：为何说ELMo依赖于特定任务的架构？**
  - **预训练和微调的设计**：
    - ELMo首先使用双向LSTM预训练一个语言模型，然后提取这个模型的中间层表示作为词的向量表示。
    - 在下游任务中，这些表示被视为特征，并被附加到特定任务的模型中。
    - 换句话说，ELMo表示是附加到任务特定模型的，而不是替代它。
  - **任务特定的结构**：
    - 当使用ELMo为下游任务（如情感分析、命名实体识别或语义角色标注）提供上下文敏感表示时，任务的原始模型结构不会被完全抛弃。
    - 相反，ELMo表示被加入到已有的任务特定模型中。
    - 例如，在一个序列标注任务中，ELMo的表示可能会被添加到传统的序列标注模型（如条件随机场）中。
  - **冻结的预训练模型**：
    - 当ELMo表示被加入到下游任务模型时，预训练的双向LSTM模型中的权重是被冻结的，不会在下游任务的训练中进行更新。
    - 这意味着下游任务需要一个额外的模型结构来进行训练和微调。
  - 这些特点使ELMo依赖于特定任务的架构。
    - 每个自然语言处理任务都可能需要一个略有不同的模型结构来最好地处理其特定的挑战，而ELMo的设计允许并鼓励这种多样性，它为不同任务提供上下文敏感的表示，但保留了原始任务特定的模型结构。
-----------

## 14.8.3 BERT：把两个最好的结合起来

如我们所见，ELMo对上下文进行双向编码，但使用特定于任务的架构；而GPT是任务无关的，但是从左到右编码上下文。BERT（来自Transformers的双向编码器表示）结合了这两个方面的优点。它对上下文进行双向编码，并且对于大多数的自然语言处理任务只需要最少的架构改变。通过使用预训练的Transformer编码器，BERT能够基于其双向上下文表示任何词元。在下游任务的监督学习过程中，BERT在两个方面与GPT相似。首先，BERT表示将被输入到一个添加的输出层中，根据任务的性质对模型架构进行最小的更改，例如预测每个词元与预测整个序列。其次，对预训练Transformer编码器的所有参数进行微调，而额外的输出层将从头开始训练。图14.8.1描述了ELMo、GPT和BERT之间的差异。

<center><img src='../img/elmo-gpt-bert.svg'></center>
<center>图14.8.1 ELMo、GPT和BERT的比较</center>

BERT进一步改进了11种自然语言处理任务的技术水平，这些任务分为以下几个大类：（1）单一文本分类（如情感分析）、（2）文本对分类（如自然语言推断）、（3）问答、（4）文本标记（如命名实体识别）。从上下文敏感的ELMo到任务不可知的GPT和BERT，它们都是在2018年提出的。概念上简单但经验上强大的自然语言深度表示预训练已经彻底改变了各种自然语言处理任务的解决方案。

在本章的其余部分，我们将深入了解BERT的训练前准备。当在第15章中解释自然语言处理应用时，我们将说明针对下游应用的BERT微调。

- **要点：**
  - **ELMo 和 GPT 的局限性**:
    - ELMo 对上下文进行双向编码，但使用了特定于任务的架构。
    - GPT 是任务无关的，但仅从左到右编码上下文。
  - **BERT 的优势**:
    - BERT 结合了 ELMo 的双向上下文编码和 GPT 的任务无关性的优点。
    - 它对上下文进行双向编码，并对大多数自然语言处理任务只需要最小的架构改动。
  - **BERT 的工作机制**:
    - 使用预训练的 Transformer 编码器，BERT 可以基于其双向上下文表示任何词元。
    - 在下游任务中，BERT 的表示会被输入到一个附加的输出层。
    - 根据任务的性质对模型进行最小的修改，例如预测每个词元或预测整个序列。
    - 对预训练的 Transformer 编码器的所有参数进行微调，而附加的输出层则从头开始训练。
  - **BERT 的成果**:
    - BERT 进一步改进了 11 种自然语言处理任务的技术水平。
    - 这些任务包括单一文本分类（如情感分析）、文本对分类（如自然语言推断）、问答和文本标记（如命名实体识别）。
  - **发展历程**:
    - 从上下文敏感的 ELMo 到任务不可知的 GPT 和 BERT，它们都是在 2018 年提出的。
    - 这种自然语言深度表示的预训练方法在概念上简单，但在实践中十分强大，已经彻底改变了各种自然语言处理任务的解决方案。

In [1]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

## 14.8.4 输入表示

- 在自然语言处理中，有些任务（如情感分析）以**单个文本作为输入**，而有些任务（如自然语言推断）以**一对文本序列作为输入**。
- BERT输入序列明确地表示单个文本和文本对。
  - 当输入为单个文本时，BERT输入序列是特殊类别词元“&lt;cls&gt;”、文本序列的标记、以及特殊分隔词元“&lt;sep&gt;”的连结。
  - 当输入为文本对时，BERT输入序列是“&lt;cls&gt;”、第一个文本序列的标记、“&lt;sep&gt;”、第二个文本序列标记、以及“&lt;sep&gt;”的连结。
- 我们将始终如一地将术语“BERT输入序列”与其他类型的“序列”区分开来。
  - 例如，一个**BERT输入序列**可以包括一个**文本序列**或两个**文本序列**。
- 为了区分文本对，根据输入序列学到的片段嵌入$\mathbf{e}_A$和$\mathbf{e}_B$分别被添加到第一序列和第二序列的词元嵌入中。
  - 对于单文本输入，仅使用$\mathbf{e}_A$。
- 下面的`get_tokens_and_segments`将一个句子或两个句子作为输入，然后返回BERT输入序列的标记及其相应的片段索引。


In [2]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    ## 0表示第一个片段，+2是为<cls>和<sep>预留位置。
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        ## 1表示第二个片段，+1是为末尾的<sep>预留位置
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [3]:
'''
（1）函数的两个参数都是list，里面保存多个字符串
（2）然后按照list将多个字符串连接成一个片段segment
（3）有几个list就有几个segments
（4）每个segment以'<cls>'开头，segment之间使用'<sep>'隔开
（5）每个segment由1个或多个token构成
（6）函数返回的segments里保存的是片段索引。0表示第一个片段的tokens，1表示第二个片段的tokens。
（7）函数可以返回一个片段，也可以返回两个片段。主要目的就是在片段前加上<cls>；
    如果有两个片段，在每个片段的末尾加上<sep>
'''
print(get_tokens_and_segments(['this','movie','is','great'],['i','like','it']))

(['<cls>', 'this', 'movie', 'is', 'great', '<sep>', 'i', 'like', 'it', '<sep>'], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1])


- BERT选择Transformer编码器作为其双向架构。
- 在Transformer编码器中常见是，位置嵌入被加入到输入序列的每个位置。
  - 然而，与原始的Transformer编码器不同，BERT使用**可学习的位置嵌入**。
- 总之， 图14.8.2表明BERT输入序列的嵌入是**词元嵌入、片段嵌入和位置嵌入的和**。
<center><img src='../img/bert-input.svg'></center>
<center>图14.8.2 BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和</center>
- 下面的`BERTEncoder`类类似于 10.7节中实现的`TransformerEncoder`类。
  - 与`TransformerEncoder`不同，`BERTEncoder`使用片段嵌入和可学习的位置嵌入。


In [8]:
#@save
class BERTEncoder(nn.Module):
    """BERT编码器"""
    ## 参数也与Transformer的Encoder差不多，没啥区别
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        ## 多了一个片段嵌入，形状是2层，对句子对中的2个句子进行嵌入编码
        ## 即对表示第1,2个句子的0，1两个字符进行嵌入编码
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        ## Base是12层，Large是24层，本书例子是2层
        for i in range(num_layers):
            ## 使用Transformer的Encoder块进行叠加
            self.blks.add_module(f"{i}", d2l.EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        ## 位置嵌入张量是3维的，该张量在本例中的形状是(1,1000,768)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))
        print('pos_embedding.shape:',self.pos_embedding.shape)
        
    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        ## 词嵌入、片段嵌入以及位置嵌入，三个嵌入相加
        
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        print('X.shape(token_embedding+segment_embedding):', X.shape)
        
        ## 位置嵌入初始值有1000行(max_len),但是只需要X的行数即可
        ## :X.shape[1]表示值取出位置嵌入的前X.shape[1]行
        ## 位置编码在批量维进行了广播。本例中从1广播到2
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        print('X.shape(+pos_embedding):', X.shape)
        
        ## 然后一层层调用编码器块，Bert small有12层:1.1亿，Bert large有24层：3.4亿
        ## 本书例子只有2层，在3090ti上跑的还行，比教材上的结果要快一点。        
        for blk in self.blks:
            X = blk(X, valid_lens)
        
        ## 编码层的输出形状(batch_size, seq_len, num_hiddens)
        ## 本例为(2, 8, 768),保存隐状态。那么BERT最终保存的应该是什么呢？
        ## 是各种隐藏层中间参数
        return X

- 假设词表大小为10000，为了演示`BERTEncoder`的前向推断，让我们创建一个实例并初始化它的参数。


In [5]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

pos_embedding.shape: torch.Size([1, 1000, 768])


----------------
- **说明:LayNorm层归一化再举例**

In [31]:
import torch
n = torch.arange(24).reshape(2,3,4).float()
## 如果是4且是最后一维，那么就是对最后一维归一化。
## 本例中对2个矩阵各自的3行进行标准化
## 归一化后的张量形状不发生变化。
ln = torch.nn.LayerNorm([4])
ln(n),n

(tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
          [-1.3416, -0.4472,  0.4472,  1.3416],
          [-1.3416, -0.4472,  0.4472,  1.3416]],
 
         [[-1.3416, -0.4472,  0.4472,  1.3416],
          [-1.3416, -0.4472,  0.4472,  1.3416],
          [-1.3416, -0.4472,  0.4472,  1.3416]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.]],
 
         [[12., 13., 14., 15.],
          [16., 17., 18., 19.],
          [20., 21., 22., 23.]]]))

In [32]:
## 对最后一维归一化，分母是标准差的有偏估计，即unbiased=False
(n[0,0]-n[0,0].mean())/(n[0,0].std(unbiased=False))

tensor([-1.3416, -0.4472,  0.4472,  1.3416])

In [33]:
##[3,4]归一化，最后2维归一化，其实就是对两个矩阵分别归一化
ln=torch.nn.LayerNorm([3,4])
ln(n)

tensor([[[-1.5933, -1.3036, -1.0139, -0.7242],
         [-0.4345, -0.1448,  0.1448,  0.4345],
         [ 0.7242,  1.0139,  1.3036,  1.5933]],

        [[-1.5933, -1.3036, -1.0139, -0.7242],
         [-0.4345, -0.1448,  0.1448,  0.4345],
         [ 0.7242,  1.0139,  1.3036,  1.5933]]],
       grad_fn=<NativeLayerNormBackward0>)

In [34]:
## 手工实现第一个矩阵归一化
(n[0]-n[0].mean())/n[0].std(unbiased=False)

tensor([[-1.5933, -1.3036, -1.0139, -0.7242],
        [-0.4345, -0.1448,  0.1448,  0.4345],
        [ 0.7242,  1.0139,  1.3036,  1.5933]])

------------

- 我们将`tokens`定义为长度为8的2个输入序列，其中每个词元是词表的索引。
- 使用输入`tokens`的`BERTEncoder`的前向推断返回编码结果，其中每个词元由向量表示，其长度由超参数`num_hiddens`定义。
- 此超参数通常称为Transformer编码器的**隐藏大小**（隐藏单元数）。


In [10]:
## 词元在词表中的索引，本例小批量有2个句子或句子对，长度为8
tokens = torch.randint(0, vocab_size, (2, 8))
## 片段，用于表示句子的表示，0表示句子对中的第一个句子，1表示第二个句子
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
## 通过BERT编码计算，返回结果是2个长度为8的句子或句子对的BERT编码，其实就是隐藏层参数
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

X.shape(token_embedding+segment_embedding): torch.Size([2, 8, 768])
X.shape(+pos_embedding): torch.Size([2, 8, 768])


torch.Size([2, 8, 768])

In [11]:
## 上面例子中的词元示例形状以及具体数据
tokens.shape, tokens

(torch.Size([2, 8]),
 tensor([[6286, 9992, 6411,   63, 1803, 6858, 5927, 1580],
         [8817, 9167, 5849, 8454, 8904, 2134, 5734,   55]]))

## 14.8.5 预训练任务
- `BERTEncoder`的前向推断给出了输入文本的每个词元和插入的特殊标记“&lt;cls&gt;”及“&lt;seq&gt;”的BERT表示。
- 接下来，我们将使用这些表示来计算预训练BERT的损失函数。
- 预训练包括以下两个任务：掩蔽语言模型和下一句预测。

### 14.8.5.1 掩蔽语言模型（Masked Language Modeling）
- 如 8.3节所示，语言模型使用**左侧**的上下文预测词元。
- 为了双向编码上下文以表示每个词元，BERT**随机掩蔽词元**并使用来自双向上下文的词元以**自监督的方式预测掩蔽词元**。
  - 此任务称为**掩蔽语言模型**。
- 在这个预训练任务中，将**随机选择15%的词元作为预测的掩蔽词元**。
- 要预测一个掩蔽词元而不使用标签作弊，一个简单的方法是总是用一个特殊的“&lt;mask&gt;”替换输入序列中的词元。
  - 但是人造特殊词元“&lt;mask&gt;”不会出现在微调中。
- **为了避免预训练和微调之间的这种不匹配，** 如果为预测而屏蔽词元（例如，在“this movie is great”中选择掩蔽和预测“great”），则在输入中将其替换为：
  -  80%时间为特殊的“&lt;mask&gt;“词元（例如，“this movie is great”变为“this movie is&lt;mask&gt;”；
  -  10%时间为随机词元（例如，“this movie is great”变为“this movie is drink”）；
  -  10%时间内为不变的标签词元（例如，“this movie is great”变为“this movie is great”）。
- 请注意，在15%的时间中，有10%的时间插入了随机词元。
  - 这种偶然的噪声鼓励BERT在其双向上下文编码中不那么偏向于掩蔽词元（尤其是当标签词元保持不变时）。
- 我们实现了下面的`MaskLM`类来预测BERT预训练的掩蔽语言模型任务中的掩蔽标记。
  - 预测使用单隐藏层的多层感知机（`self.mlp`）。
  - 在前向推断中，它需要两个输入：<b>`BERTEncoder`的编码结果和用于预测的词元位置</b>。
  - 输出是这些位置的预测结果。

In [12]:
#@save
'''
以mlm_positions=torch.tensor([[1, 5, 2], [6, 1, 5]])为例。
'''
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    ## vocab_size=10000,num_hiddens=768
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        '''
        输入(2, 3, 768)，输出(2,3,10000),最后输出是6个tokens在词表中的概率分布。
        归根到底就是一种学习和预测。主要目的是反向传播和优化参数(全连接层的权重与偏置)。
        '''
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    ## pred_positions的形状(2,3)
    def forward(self, X, pred_positions):
        
        ## 预测位置的个数:3
        ## 在2个批量上各自预测3个masked tokens
        ## num_pred_positions = 3, 即pred_positions的第二维形状
        num_pred_positions = pred_positions.shape[1]
        
        ## 预测位置的形状变形为一维数组
        ## 从[[1, 5, 2], [6, 1, 5]]到[1, 5, 2, 6, 1, 5]
        pred_positions = pred_positions.reshape(-1)
        
        ## batch_size=2,X:(2,8,768)
        batch_size = X.shape[0]
        
        ## batch_idx=[0,1]
        batch_idx = torch.arange(0, batch_size)
        
        # 假设batch_size=2，num_pred_positions=3
        # 那么batch_idx是np.array([0,0,0,1,1,1])
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)        
        '''
        [0,0,0,1,1,1]与[1, 5, 2, 6, 1, 5]作为行索引和列索引，从X里获取mask的位置。
        一共6个tokens被掩码，然后将之赋给masked_X。这6个tokens是真实的tokens。
        掩码的结果是(6,768)，即6个tokens的BERT编码
        '''
        masked_X = X[batch_idx, pred_positions]
        ## 将masked_X的shape从(6, 768)变形为(2, 3, 768)
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        '''
        然后将masked_X传送给多层感知机，预训练MLM模型。
        注意：pred_positions是进行预训练任务时，
        随机选择句子或句子对中15%的词元作为预测的掩蔽词元，
        然后又按照3种比例对这15%掩码词元进行处理。
        '''
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

In [13]:
## repeat_interleave的用法
r = torch.tensor([0,1])
torch.repeat_interleave(r,3)

tensor([0, 0, 0, 1, 1, 1])

- 为了演示`MaskLM`的前向推断，我们创建了其实例`mlm`并对其进行了初始化。
- 回想一下，来自`BERTEncoder`的正向推断`encoded_X`表示2个BERT输入序列。
- 我们将`mlm_positions`定义为在`encoded_X`的任一输入序列中预测的3个**指示**（？）。
- `mlm`的前向推断返回`encoded_X`的所有掩蔽位置`mlm_positions`处的预测结果`mlm_Y_hat`。
- 对于每个预测，结果的大小等于词表的大小。


In [14]:
## 10000,768
mlm = MaskLM(vocab_size, num_hiddens)
'''
这个应该是序列中的被掩码tokens的位置索引。应该是按照前文中三种方式生成15%掩码后的词元索引。
即80%的<mask>，10%随机词元以及10%的原真实词元。都以掩码词元在词表中的索引表示，此处是模拟数据。
'''
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
'''
encoded_X:(2, 8, 768), mlm_positions:(2, 3)。
第一个参数表示2个长度为8的句子，然后768表示隐藏层编码输出的长度。
mlm_positions表示在句子里的进行掩码操作的索引位置。即2个长度为8的句子对
中的各自的掩码索引：1, 5, 2表示第一个句子中索引为1, 5, 2的地方为掩码<mask>，
[6, 1, 5]类似。
'''
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

In [60]:
## 掩码预测位置的索引，变形为一维数组
## 本例中对小批量数据的一种处理技巧
mlm_positions.reshape(-1)

tensor([1, 5, 2, 6, 1, 5])

- 通过掩码下的预测词元`mlm_Y`的真实标签`mlm_Y_hat`，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。


In [62]:
## 这个是真实标签，也就是被预测masked tokens在词表中的索引
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
## 计算交叉熵损失
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape, mlm_l

(torch.Size([6]),
 tensor([9.0483, 8.2577, 7.9532, 9.6828, 8.9827, 9.6567],
        grad_fn=<NllLossBackward0>))

### 14.8.5.2 下一句预测（Next Sentence Prediction）

- 尽管掩蔽语言建模能够编码双向上下文来表示单词，但它不能显式地建模文本对之间的逻辑关系。
- 为了帮助理解两个文本序列之间的关系，BERT在预训练中考虑了一个二元分类任务—— **下一句预测**。 
- 在为预训练生成句子对时:
  - 有一半的时间它们确实是标签为“真”的连续句子；
  - 在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。
- 下面的`NextSentencePred`类使用单隐藏层的多层感知机来预测第二个句子是否是BERT输入序列中第一个句子的下一个句子。
- 由于Transformer编码器中的自注意力，特殊词元“&lt;cls&gt;”的BERT表示已经对输入的两个句子进行了编码。
- 因此，多层感知机分类器的输出层（`self.output`）以`X`作为输入，其中`X`是多层感知机隐藏层的输出，而MLP隐藏层的输入是编码后的“&lt;cls&gt;”词元。


In [64]:
#@save
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        ## 这里的2表示的是由两个元素构成，那么如何表达句子是否为前后句子对？
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        ## 输出形状：(batchsize, 2)
        return self.output(X)

- 我们可以看到，`NextSentencePred`实例的前向推断返回每个BERT输入序列的**二分类预测。**


In [66]:
## encoded_X:(2,8,768)->(2,6144)
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
'''
本例中的值为(2,6144)，然后输入到NextSentencePred的MLP进行计算。
(2,6144)@(2,2)=(2,2)
'''
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
'''
X的形状为(2,8)，应该表示的两个句子对。每个句子对长度为8，也就是每对句子中的两个句子的tokens加起来是8。
当然里面还填充了<cls><sep>等特殊token。
可不可以这样理解？[0.0414, 0.4662]表示不是前后句子对，而[0.8990, 0.8142]为前后句子对？
'''
nsp_Y_hat.shape, nsp_Y_hat

(torch.Size([2, 2]),
 tensor([[0.0414, 0.4662],
         [0.8990, 0.8142]], grad_fn=<AddmmBackward0>))

In [68]:
encoded_X.shape

torch.Size([2, 6144])

-------------
- **说明：flatten的用法**

In [79]:
f=torch.arange(24).reshape(2,3,4)
## start_dim不设值，展平成1维数组，按照行拼接
f,f.flatten()

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],
 
         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23]))

In [74]:
f.flatten(start_dim=0)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])

In [75]:
## 从第2轴开始展平，就是将2个矩阵分别按行拼接
f.flatten(start_dim=1)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]])

In [78]:
## 第3轴展平则没有变化
f.flatten(start_dim=2)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

-------------

- 还可以计算两个二元分类的交叉熵损失。


In [82]:
## [0,1]表示二者不是前后句子对吗？
## 使用随机句子对来进行训练
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape,nsp_l

(torch.Size([2]), tensor([0.9279, 0.7364], grad_fn=<NllLossBackward0>))

- 值得注意的是，上述两个预训练任务中的所有标签都可以从预训练语料库中获得，而无需人工标注。
- 原始的BERT已经在图书语料库和英文维基百科的连接上进行了预训练。
- 这两个文本语料库非常庞大：它们分别有8亿个单词和25亿个单词。

## 14.8.6 整合代码
- 在预训练BERT时，**最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。**
  - 即将masked tokens与句子预测的损失函数结合来训练模型。</font>）
- 现在我们可以通过实例化三个类`BERTEncoder`、`MaskLM`和`NextSentencePred`来定义`BERTModel`类。
- 前向推断返回编码后的BERT表示`encoded_X`、掩蔽语言模型预测`mlm_Y_hat`和下一句预测`nsp_Y_hat`。

In [14]:
#@save
class BERTModel(nn.Module):
    """BERT模型"""
    ## 初始化输入的参数很多，15个参数，这个的确有难度，不方便
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        ## 编码器
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        ## masked tokens的预测损失
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        ## 句子对预测损失
        self.nsp = NextSentencePred(nsp_in_features)

    ## 调用时使用的序列、片段、有效长度和预测位置
    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        
        ## 是预测masked tokens(MLM)，还是句子对(NSP)？在这儿可以判定
        ## MLM有掩码词元的位置，而NSP则没有。
        if pred_positions is not None:
            ## MLM的预测
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
            
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        '''
        此处尤为关键。
        在下一句预测时，最后一层全连接层的输入只是经过BERTEncoder返回的序列编码
        中每个序列(句子或句子对)的第一个token即<cls>的BERT编码，其他都舍弃了。
        [:, 0, :]表示所有批量第一个字符的BERT编码，其形状是(batch_size,1,num_hiddens)
        '''
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        '''
        返回形状：
        encoded_X: (batch_size,seq_len, num_hiddens)
        mlm_Y_hat: (batch_size,seq_len, vacob_len)
        nsp_Y_hat: (batch_size,2)
        '''
        return encoded_X, mlm_Y_hat, nsp_Y_hat

## 小结

* word2vec和GloVe等词嵌入模型与上下文无关。它们将相同的预训练向量赋给同一个词，而不考虑词的上下文（如果有的话）。它们很难处理好自然语言中的一词多义或复杂语义。
* 对于上下文敏感的词表示，如ELMo和GPT，词的表示依赖于它们的上下文。
* ELMo对上下文进行双向编码，但使用特定于任务的架构（然而，为每个自然语言处理任务设计一个特定的体系架构实际上并不容易）；而GPT是任务无关的，但是从左到右编码上下文。
* BERT结合了这两个方面的优点：它对上下文进行双向编码，并且需要对大量自然语言处理任务进行最小的架构更改。
* BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和。
* 预训练包括两个任务：掩蔽语言模型和下一句预测。前者能够编码双向上下文来表示单词，而后者则显式地建模文本对之间的逻辑关系。